# Fed-BioMed Researcher - Loading breakpoints only

This examples demonstrates how to use a breakpoint from a previously run experiment.

Thus you need to :
    * have previously started an experiment with `save_breakpoints=True` such as in `general-resume-experiment`
    * the experiment must at least have completed a round, so that it saves a breakpoint
    * ideally, you interrupt the experiment before completion so that not all rounds have completed, to simulate a failure during experiment
    * then you restart the same nodes as for the experiment, with same dataset available
    * and you run this notebook that will use the last breakpoint and complete the experiment

## Resume an experiment

**To load the latest breakpoint of the latest experiment**

Run :
`Experiment.load_breakpoint()`. It reloads latest breakpoint, and will bypass `search` method

and then use `.run` method as you would do with an existing experiment.

**To load a specific breakpoint** specify breakpoint folder.

- absolute path: use `Experiment.load_breakpoint("${FEDBIOMED_DIR}/var/experiments/Experiment_xxxx/breakpoint_yyyy)`. Replace `xxxx` and `yyyy` by the real values.
- relative path from a notebook: a notebook is running from the `${FEDBIOMED_DIR}/notebooks` directory
so use `Experiment.load_breakpoint("../var/experiments/Experiment_xxxx/breakpoint_yyyy)`. Replace `xxxx` and `yyyy` by the real values.
- relative path from a script: if launching the script from the
  ${FEDBIOMED_DIR} directory (eg: `python ./notebooks/general-breakpoint-save-resume.py`) then use a path relative to the current directory eg: `Experiment.load_breakpoint("./var/experiments/Experiment_xxxx/breakpoint_yyyy)`


In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment

loaded_exp = Experiment.load_breakpoint('/home/ybouilla/Documents/github/fedbiomed/var/experiments/Experiment_0003/breakpoint_0000')

Verify which experiment was loaded (using heuristic to guess last experiment if no breakpoint was specified)

In [ ]:
print(f'Loaded experimentation folder: {loaded_exp.experimentation_folder()}')
print(f'Loaded experiment path: {loaded_exp.experimentation_path()}')

Continue training for the experiment loaded from breakpoint. If you ran all the rounds and load the last breakpoint, there won't be any more round to run.

In [ ]:
loaded_exp.run(rounds=2, increase=True)

Save trained model to file

In [ ]:
loaded_exp.training_plan().export_model('./trained_model')

Example of displaying some of the experiment results

In [ ]:
rounds = loaded_exp.round_current()

print("\nList the training rounds : ", loaded_exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = loaded_exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).
For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", loaded_exp.aggregated_params().keys())


print("\nAccess the federated params for the last training round :")
print("\t- parameter data: ", loaded_exp.aggregated_params()[rounds - 1]['params'].keys())
